### 1 - Importar Arquivos e Bibliotecas

In [73]:
# Importar bibliotecas
import pandas as pd
from pathlib import Path
import yagmail
from email_username_password import password, email
from pretty_html_table import build_table
import numpy as np

In [74]:
#importar bases de dados
df_emails = pd.read_excel(r"Bases de Dados/Emails.xlsx")
df_lojas = pd.read_csv(r"Bases de Dados/Lojas.csv", sep=";", encoding="latin1")
df_vendas = pd.read_excel(r"Bases de Dados/Vendas.xlsx")

df_emails.dropna(how="all")

display(df_emails.head())
display(df_lojas.head())
display(df_vendas.head())

,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,edsoncarvalhointuria+helena@gmail.com
1,Shopping Midway Mall,Alice,edsoncarvalhointuria+alice@gmail.com
2,Norte Shopping,Laura,edsoncarvalhointuria+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,edsoncarvalhointuria+manuela@gmail.com
4,Shopping União de Osasco,Valentina,edsoncarvalhointuria+valentina@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


In [75]:
# Criar variaveis uteis
diario, anual = "Diario", "Anual"

### 2 - Criar uma tabela para cada Loja e calcular o faturamento, variedade de produtos e ticket médio por loja (anual e diario)

In [76]:
# Criar funções
def faturamento_dia(dicionario_lojas:dict, loja: str, is_dia:bool):
    return np.nan_to_num(dicionario_lojas[loja].loc[is_dia,"Valor Final"].sum())
def faturamento_ano(dicionario_lojas:dict, loja: str, is_ano:bool):
    return np.nan_to_num(dicionario_lojas[loja].loc[is_ano,"Valor Final"].sum())
    

def var_produtos_dia(dicionario_lojas:dict, loja: str, is_dia:bool):
    return np.nan_to_num(len(dicionario_lojas[loja].loc[is_dia,"Produto"].unique()))
def var_produtos_ano(dicionario_lojas:dict, loja: str, is_ano:bool):
    return np.nan_to_num(len(dicionario_lojas[loja].loc[is_ano,"Produto"].unique()))


def ticket_medio_dia(faturamento_do_dia: float, dicionario_lojas:dict, loja: str, is_dia:bool):
    return np.nan_to_num(faturamento_do_dia/len(dicionario_lojas[loja].loc[is_dia,"Código Venda"].unique()))
def ticket_medio_ano(faturamento_do_ano: float, dicionario_lojas:dict, loja: str, is_ano:bool):
    return np.nan_to_num(faturamento_do_ano/len(dicionario_lojas[loja].loc[is_ano,"Código Venda"].unique()))

In [77]:
#Criar variaveis
dict_dfs_lojas = {}
dict_faturamento_lojas = {diario:{}, anual:{}} 
dict_variedade_produtos = {diario:{}, anual:{}} 
dict_ticket_medio = {diario:{}, anual:{}}


In [78]:
# Mesclando tabela de lojas com vendas
df_vendas = df_vendas.merge(df_lojas, on="ID Loja")

# Extraindo valor das planilhas
for loja in df_lojas["Loja"]: 
    dict_dfs_lojas[loja] = df_vendas[df_vendas["Loja"] == loja] # Dividir o dataframe de vendas por loja

    # Definindo variaveis para analise
    ultima_data = df_vendas["Data"].max() #Pegando a última data
    is_ultimo_dia = dict_dfs_lojas[loja]["Data"] == ultima_data #Filtrando o dataframe pelo último dia
    is_ultimo_ano = dict_dfs_lojas[loja]["Data"].dt.year == ultima_data.year #Filtrando o dataframe pelo último ano

    #Adicionando os faturamentos diarios e anuais no dicionario faturamento_loja
    dict_faturamento_lojas[diario][loja] = faturamento_dia(dict_dfs_lojas, loja=loja, is_dia=is_ultimo_dia)
    dict_faturamento_lojas[anual][loja] = faturamento_ano(dict_dfs_lojas, loja=loja, is_ano=is_ultimo_ano)

    #Adicionando a variedade de produtos diarios e anuais no dicionario variedade_produtos
    dict_variedade_produtos[diario][loja] = var_produtos_dia(dict_dfs_lojas, loja=loja, is_dia=is_ultimo_dia)
    dict_variedade_produtos[anual][loja] = var_produtos_ano(dict_dfs_lojas, loja=loja, is_ano=is_ultimo_ano)

    #Adicionando o ticket médio diario e anual no dicionario ticket_medio
    dict_ticket_medio[diario][loja] = ticket_medio_dia(dict_faturamento_lojas[diario][loja], dict_dfs_lojas, loja=loja, is_dia=is_ultimo_dia)
    dict_ticket_medio[anual][loja] = ticket_medio_ano(dict_faturamento_lojas[anual][loja], dict_dfs_lojas, loja=loja, is_ano=is_ultimo_ano)


C:\Users\Edson\AppData\Local\Temp\ipykernel_17540\3661060001.py:15: RuntimeWarning: invalid value encountered in scalar divide
  return np.nan_to_num(faturamento_do_dia/len(dicionario_lojas[loja].loc[is_dia,"Código Venda"].unique()))


### 3 - Salvar a planilha na pasta de backup

In [79]:
# Definir função
def criar_backup(dir:Path, lista_lojas:dict, data):
    diretorio = Path(dir)
    # criando o diretório, caso ele não exista
    if not diretorio.exists(): 
        diretorio.mkdir()

    # Criando subpastas de cada loja e salvando os dataframes no formato xlsx
    for loja in lista_lojas: #Para cada loja no dataframe das lojas, pegar os valores unicos da coluna Loja
        arquivo_dir = Path(diretorio, loja)
        if not arquivo_dir.exists():
            arquivo_dir.mkdir()
        lista_lojas[loja].to_excel(rf"{arquivo_dir}/{data}_{loja}.xlsx", index=False)

In [80]:
criar_backup(dir="Backup Arquivos Lojas", lista_lojas=dict_dfs_lojas, data=str(ultima_data.strftime("%d_%m_%Y")))

### 4 - Mandar e-mail para os gerentes

In [81]:
# Definir modelo de email 
modelo_email = '''Prezado(a), NOME_
Segue o resultado do dia DIA_:
</br>'''

# inclui chaves na tabela para usar o format
with open("Bases de Dados/table.html", "r", encoding="utf-8") as table:
    tabela = table.read().replace("\n", "").replace("  ", "")
# '''<table style="width: 70%; border-collapse: collapse; margin: 25px 0; font-size: 18px; text-align: left;"><thead><tr style="background-color: #009879; color: white; text-align: left; font-weight: bold;"><th style="padding: 12px 15px; border: 1px solid #ddd;"></th><th style="padding: 12px 15px; border: 1px solid #ddd;">Valor Dia</th><th style="padding: 12px 15px; border: 1px solid #ddd;">Meta Dia</th><th style="padding: 12px 15px; border: 1px solid #ddd;">Cenário Dia</th></tr></thead><tbody><tr style=border-bottom: 1px solid #ddd;"><th style="padding: 12px 15px; border: 1px solid #ddd;">Faturamento</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th></tr><tr style=border-bottom: 1px solid #ddd;><th style="padding: 12px 15px; border: 1px solid #ddd;">Diversidade Prods</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th></tr><tr style=border-bottom: 1px solid #ddd;><th style="padding: 12px 15px; border: 1px solid #ddd;">Ticket Médio</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th><th style="padding: 12px 15px; border: 1px solid #ddd;">{}</th></tr></tbody></table></br>'''

att = '''</br>Atenciosamente,
Edson C.'''

In [82]:
username = yagmail.SMTP(email, password)
metas = {diario:{"Faturamento":1000, "Diversidade de Produtos":4, "Ticket Médio":500},
         anual:{"Faturamento":1650000, "Diversidade de Produtos":120, "Ticket Médio":500}}

# Modificando e enviando o e-mail para o gerente de cada loja
for i in df_emails.index:
    loja = df_emails.loc[i, "Loja"]# Verificar se o valor atual da coluna Loja é igual a Diretoria
    if loja == "Diretoria":
        diretoria = df_emails.loc[i, "E-mail"]
        continue

    # Criar varivel para edição do corpo do email
    email_body = modelo_email

    # #Criar email--------------------------
    email_body = email_body.replace("NOME_", df_emails.loc[i, "Gerente"]) # Alterando o placeholder "NOME_" pelo nome do gerente
    ultima_data = df_vendas["Data"].max().strftime("%d/%m/%Y") #Pegando a ultima data da loja atual e transformando em string
    email_body = email_body.replace("DIA_", ultima_data) #Substituindo o placeholder "DIA_" pela ultima data
    
    for periodo in [diario, anual]:
        tabela_body = tabela # Criar varivel para edição do corpo do email
        if periodo == anual:
            tabela_body = tabela_body.replace("Dia", "Ano")
        # Incluido valores na tabela, usando o format
        valor_faturamento = dict_faturamento_lojas[periodo][loja]
        meta_faturamento = metas[periodo]["Faturamento"]
        qtd_variedade = dict_variedade_produtos[periodo][loja]
        meta_variedade = metas[periodo]["Diversidade de Produtos"]
        valor_ticket = dict_ticket_medio[periodo][loja]
        meta_ticket = metas[periodo]["Ticket Médio"]

        # As Colunas São: Valor Dia, Meta Dia e Cenário Dia
        # A ordem dos dados são: Faturamento, diversidadede de produtos e Ticket Médio
                                        #Faturamento   
        tabela_body = tabela_body.format(valor_faturamento, meta_faturamento, '<font color="green">◙</font>' if valor_faturamento >= meta_faturamento else '<font color="red">◙</font>',
                                        #Diversidade de Produtos
                                        qtd_variedade, meta_variedade, '<font color="green">◙</font>' if qtd_variedade >= meta_variedade else '<font color="red">◙</font>',
                                        #Ticket Médio
                                        valor_ticket, meta_ticket, '<font color="green">◙</font>' if valor_ticket >= meta_ticket else '<font color="red">◙</font>')
        email_body += tabela_body
    email_body+= att

    # # Encaminhar o email------------------------------
    subject = f"One page dia {ultima_data} loja {loja}"
    to = df_emails.loc[i, "E-mail"]
    attacments = Path("Backup Arquivos Lojas", loja, f"{ultima_data.replace("/", "_")}_{loja}.xlsx")
    username.send(to=to, subject=subject, contents=email_body, attachments=[attacments])
    
    

### 5 - Mandar email para a diretoria com os rankings das loja

In [83]:
email_body = f"""Prezados,
\nSegue abaixo o ranking das lojas, referente a ultima data ({ultima_data}):
"""

for periodo in [diario, anual]:
    email_body += f"<h2>{periodo}</h2>"
    # Organizando o dataframe do menor para o maior
    tabela_body = (pd.DataFrame(dict_faturamento_lojas[periodo].items(), columns=["Loja", "Valor"])).sort_values("Valor", ascending=False, ignore_index=True)
    tabela_body.to_excel(rf"Backup Arquivos Lojas\{ultima_data.replace("/", "_")}_ranking {periodo}.xlsx")
    email_body += build_table(tabela_body[:5], 'yellow_light').replace("\n", "").replace("  ", "") + "\n"
email_body += att

# Encaminhando o email
subject = f"Ranking lojas"
to= diretoria
attacments = [arquivo for arquivo in Path("Backup Arquivos Lojas").iterdir() if "xlsx" in arquivo.name]
username.send(to=to, subject=subject, contents=email_body, attachments=attacments)

{}